# Librerias

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('./pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Strategy
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
from backtesting import Strategy
from datetime import datetime
import MetaTrader5 as mt5
import itertools
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz
from utils import walk_forward
from utils import plot_full_equity_curve
import numpy as np
from sklearn.linear_model import LinearRegression


import random
random.seed(42)

# Obtener datos

In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:
# tickers = [
#     'NTESm',
#  'NFLXm',
#  'Vm',
#  'ADPm',
#  'COSTm',
#  'EBAYm',
#  'AMGNm',
#  'AAPLm',
#  'WFCm',
#  'TMUSm'
#  ]

tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Stocks_group' in symbol.path and symbol.spread < 15]
# tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Stocks_group' in symbol.path]
# tickers = [ticker for ticker in tickers if ticker == 'EURUSDm']
print(tickers)

['AAPLm', 'AMZNm', 'ATVIm', 'BACm', 'BMYm', 'Cm', 'CMCSAm', 'CSCOm', 'CSXm', 'CVSm', 'EBAYm', 'GILDm', 'INTCm', 'JNJm', 'KOm', 'MDLZm', 'MOm', 'MRKm', 'MSm', 'NKEm', 'PEPm', 'PFEm', 'PGm', 'PYPLm', 'SBUXm', 'Tm', 'TSLAm', 'VZm', 'WMTm', 'XOMm', 'AMCm', 'BBm', 'BBBYm', 'BEKEm', 'BILIm', 'BRQSm', 'BYNDm', 'CANm', 'FTNTm', 'IQm', 'JDm', 'LIm', 'NIOm', 'RLXm', 'TIGRm', 'TMEm', 'TSMm', 'VIPSm', 'XPEVm', 'YUMCm']


In [5]:
intervals = [
    # mt5.TIMEFRAME_M1,
    # mt5.TIMEFRAME_M2,
    # mt5.TIMEFRAME_D1,
    # mt5.TIMEFRAME_M3,
    mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_M5,
    # mt5.TIMEFRAME_M15,
    # mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_H2,
    # mt5.TIMEFRAME_M30,
]

parameter_combinations = list(itertools.product(
    tickers, intervals
))

symbols = {}

print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# Establecer conexión con el terminal de MetaTrader 5
if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
utc_from = datetime(2010, 10, 1, tzinfo=timezone)
utc_to = datetime(2024, 9, 1, tzinfo=timezone)

for ticker, interval in parameter_combinations:
    print(ticker)
    # Obtener las tasas históricas
    rates = mt5.copy_rates_range(ticker, interval, utc_from, utc_to)
    
    # Crear DataFrame con las tasas
    df = pd.DataFrame(rates)
    
    # Convertir el tiempo de segundos a formato datetime
    df['time'] = pd.to_datetime(df['time'], unit='s')


    # Renombrar columnas para el ticker principal
    df = df.rename(columns={
        'time': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'tick_volume': 'Volume'
    }).set_index('Date')


    if ticker not in symbols.keys():
        symbols[ticker] = {}
        symbols[ticker][interval] = {}

    symbols[ticker][interval] = df


# Cerrar la conexión con MetaTrader 5
mt5.shutdown()

# Mostrar el DataFrame final

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AAPLm
AMZNm
ATVIm
BACm
BMYm
Cm
CMCSAm
CSCOm
CSXm
CVSm
EBAYm
GILDm
INTCm
JNJm
KOm
MDLZm
MOm
MRKm
MSm
NKEm
PEPm
PFEm
PGm
PYPLm
SBUXm
Tm
TSLAm
VZm
WMTm
XOMm
AMCm
BBm
BBBYm
BEKEm
BILIm
BRQSm
BYNDm
CANm
FTNTm
IQm
JDm
LIm
NIOm
RLXm
TIGRm
TMEm
TSMm
VIPSm
XPEVm
YUMCm


True

In [6]:
max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



{16388: Timestamp('2021-10-27 16:00:00')}

# WFO

In [7]:
class EndOfMonth(Strategy):
 
    risk=1
    day_to_buy = 25
    percentage_price_sl = 5
    
    def init(self):
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=200
        )

        self.rsi_2 = self.I(
            ta.RSI, self.data.Close, 2
        )
        
    def next(self):
        
        today = self.data.index[-1]

        if self.position:
            first_trade = self.trades[0]
            time_in_position = (today - first_trade.entry_time)
            time_in_position = time_in_position.days

            if self.position.is_long:
                if self.rsi_2 > 90:
                    self.position.close()

        else:
            # two_days_bearish = self.data.Close[-1] < self.data.Close[-2] and self.data.Close[-2] < self.data.Close[3]
                                                                 
            # es el dia de compra, el precio esta por encima de la sma
            if today.day_of_week == self.day_to_buy and self.data.Close[-1] > self.sma[-1]:
                
                sl_price = self.data.Close[-1] - self.data.Close[-1] *  (self.percentage_price_sl / 100)
                
                self.buy(
                    size=self.risk/100,
                    sl=sl_price
                )

In [8]:

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    

stats_per_symbol = {}

strategy = EndOfMonth

wfo_performance = pd.DataFrame()
for ticker, intervals in symbols.items():
    for interval in intervals:
        print(ticker, interval)
        
        lookback_bars = 1000
        validation_bars = 250
        warmup_bars = 200

        params = {
            'day_to_buy': list(np.arange(0, 5, 1)),
            'percentage_price_sl': list(np.arange(2, 15, 2)),
            'maximize': optim_func_2
        }

        try:
            wfo_stats = walk_forward(
                strategy,
                symbols[ticker][interval], 
                lookback_bars=lookback_bars,
                validation_bars=validation_bars,
                warmup_bars=warmup_bars, 
                params=params,
                commission=7e-4, 
                margin=1/30, 
                cash=100_000,
                verbose=False
            )
            
            stats_per_symbol[ticker] = {}
            stats_per_symbol[ticker][interval] = wfo_stats
            
            df_equity = wfo_stats['_equity']
            
            plot_full_equity_curve(df_equity)
            
            # Calculo el stability ratio
            x = np.arange(df_equity.shape[0]).reshape(-1, 1)
            reg = LinearRegression().fit(x, df_equity.Equity)
            stability_ratio = reg.score(x, df_equity.Equity)
            

            # Extraigo metricas
            df_stats = pd.DataFrame({
                'strategy':[strategy.__name__],
                'ticker':[ticker],
                'interval':[interval],
                'stability_ratio':[stability_ratio],
                'return':[wfo_stats['Return [%]']],
                'final_eq':[wfo_stats['Equity Final [$]']],
                'drawdown':[wfo_stats['Max. Drawdown [%]']],
                'drawdown_duration':[wfo_stats['Max. Drawdown Duration']],
                'win_rate':[wfo_stats['Win Rate [%]']], 
                'sharpe_ratio':[wfo_stats['Sharpe Ratio']],
                'trades':[wfo_stats['# Trades']],
                'avg_trade_percent':[wfo_stats['Avg. Trade [%]']],
                'exposure':[wfo_stats['Exposure Time [%]']],
                'final_equity':[wfo_stats['Equity Final [$]']],
                'Duration':[wfo_stats['Duration']],

            })

            wfo_performance = pd.concat([wfo_performance, df_stats])
        
        except:
            print(f'No se pudo ejecutar para el ticker {ticker}')

wfo_performance['return/dd'] = wfo_performance['return'] / -wfo_performance['drawdown']
wfo_performance['drawdown'] = -wfo_performance['drawdown']
wfo_performance['custom_metric'] = (wfo_performance['return'] / (1 + wfo_performance.drawdown)) * np.log(1 + wfo_performance.trades)

wfo_performance.drawdown_duration = pd.to_timedelta(wfo_performance.drawdown_duration)
wfo_performance.drawdown_duration = wfo_performance.drawdown_duration.dt.days

wfo_performance.sort_values(by='return/dd', ascending=False)

AAPLm 16388


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

YUMCm 16388


  0%|          | 0/9 [00:00<?, ?it/s]

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,EndOfMonth,WMTm,16388,0.843852,2.294133,102294.133273,0.343069,169,83.333333,4.520653,12,0.630197,3.930911,102294.133273,1038 days 00:00:00,6.687087,4.381260
0,EndOfMonth,TMEm,16388,0.778193,7.477882,107477.881910,1.955288,164,70.000000,2.310993,20,1.190781,8.278737,107477.881910,1038 days 00:00:00,3.824439,7.703674
0,EndOfMonth,AMZNm,16388,0.723161,9.817435,109817.435291,3.129378,177,81.818182,1.282735,55,0.552839,9.619406,109817.435291,1038 days 00:00:00,3.137184,9.570116
0,EndOfMonth,CSXm,16388,0.900598,2.363208,102363.208490,0.769559,176,85.714286,2.275140,14,0.550389,7.921382,102363.208490,1038 days 00:00:00,3.070861,3.616544
0,EndOfMonth,BILIm,16388,0.000040,10.902147,110902.147305,4.212783,287,50.000000,1.432662,32,1.011183,4.660111,110902.147305,1038 days 00:00:00,2.587873,7.312685
0,EndOfMonth,TSMm,16388,0.847993,9.172229,109172.229306,3.712149,128,67.441860,1.226813,43,0.661575,9.255364,109172.229306,1038 days 00:00:00,2.470868,7.365950
0,EndOfMonth,BMYm,16388,0.141958,0.904601,100904.601174,0.612841,165,50.000000,1.176066,4,0.742868,1.310304,100904.601174,1038 days 00:00:00,1.476078,0.902692
0,EndOfMonth,BACm,16388,0.670250,4.089833,104089.832624,3.592255,163,67.500000,0.860678,40,0.316932,9.413255,104089.832624,1038 days 00:00:00,1.138514,3.307283
0,EndOfMonth,VIPSm,16388,0.739558,4.196803,104196.802859,3.899833,177,73.333333,1.264153,15,0.881475,8.993448,104196.802859,1038 days 00:00:00,1.076149,2.374777
0,EndOfMonth,FTNTm,16388,0.698804,2.225158,102225.158354,3.032315,144,76.923077,0.667603,13,0.535660,7.267905,102225.158354,1038 days 00:00:00,0.733815,1.456315


In [10]:
wfo_performance[
    (wfo_performance.stability_ratio > 0.7)
    & (wfo_performance['return/dd'] > 2)
]

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,EndOfMonth,AMZNm,16388,0.723161,9.817435,109817.435291,3.129378,177,81.818182,1.282735,55,0.552839,9.619406,109817.435291,1038 days,3.137184,9.570116
0,EndOfMonth,CSXm,16388,0.900598,2.363208,102363.208490,0.769559,176,85.714286,2.275140,14,0.550389,7.921382,102363.208490,1038 days,3.070861,3.616544
0,EndOfMonth,WMTm,16388,0.843852,2.294133,102294.133273,0.343069,169,83.333333,4.520653,12,0.630197,3.930911,102294.133273,1038 days,6.687087,4.381260
0,EndOfMonth,TMEm,16388,0.778193,7.477882,107477.881910,1.955288,164,70.000000,2.310993,20,1.190781,8.278737,107477.881910,1038 days,3.824439,7.703674
0,EndOfMonth,TSMm,16388,0.847993,9.172229,109172.229306,3.712149,128,67.441860,1.226813,43,0.661575,9.255364,109172.229306,1038 days,2.470868,7.365950
